In [33]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

file_path = 'dataset2.dta'



In [34]:

data = pd.read_stata(file_path, convert_categoricals=False)

##print(data.head())
umbral_nulos = 0.5
data = data.dropna(thresh=len(data) * (1 - umbral_nulos), axis=1)
# Eliminar columnas con tipo 'Timestamp' y tipo string
data = data.select_dtypes(exclude=['datetime', 'object'])
# Eliminar filas con valores nulos en la columna objetivo 'ytot'
data = data.dropna(subset=['ytot'])
# Rellenar valores nulos en otras columnas con la media de la columna
data = data.fillna(data.mean())

ss = MinMaxScaler()

ss.fit(data)

print(data)

data_normalized = ss.transform(data)




C:\Users\Gavil\AppData\Local\Temp\ipykernel_26308\1043139644.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  data = pd.read_stata(file_path, convert_categoricals=False)


        id_vivienda      folio  id_persona  region  area  cod_upm  nse  \
0           1000901  100090101           1      16     2    10009    4   
1           1000901  100090101           2      16     2    10009    4   
2           1000901  100090101           3      16     2    10009    4   
3           1000902  100090201           1      16     2    10009    4   
4           1000902  100090201           2      16     2    10009    4   
...             ...        ...         ...     ...   ...      ...  ...   
202226      4585804  458580401           2       9     1    45858    3   
202227      4585804  458580401           3       9     1    45858    3   
202228      4585807  458580701           1       9     1    45858    3   
202229      4585807  458580701           2       9     1    45858    3   
202230      4585807  458580701           3       9     1    45858    3   

        estrato  hogar  expr  ...  men18c  may60c  tipohogar  tot_hog  \
0       1630324      1    43  ...     

In [35]:
print(data_normalized)

print(data['y0101h'])
w = []

correlacion = data['y0101h'].corr(data['edad'])


for j in range(8):
    mayor = -1
    incertacion=""
    for i in data.columns:
        correlacion = data['y0101h'].corr(data[i])
        if correlacion > mayor and i not in w and i!= 'y0101h' and 'y' not in i:
            mayor = correlacion
            incertacion = i
    w.append(incertacion)
print(w)












[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.07692308 ... 0.         0.         0.        ]
 [0.         0.         0.15384615 ... 0.         0.         0.        ]
 ...
 [1.         1.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.07692308 ... 0.         0.         0.        ]
 [1.         1.         0.15384615 ... 0.         0.         0.        ]]
0               0
1               0
2               0
3               0
4               0
           ...   
202226    3400000
202227    3400000
202228    1000000
202229    1000000
202230    1000000
Name: y0101h, Length: 202231, dtype: int32


c:\Users\Gavil\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Gavil\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


['dau', 'qaut', 'dautr', 'qautr', 'n_ocupados', 'v19', 'esc', 'v13_propia']


In [36]:

y = data['y0101h']
X = data[w]


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [39]:
# Hacer predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)


In [40]:
from sklearn.metrics import r2_score
# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación R^2:", r2)


Coeficiente de determinación R^2: 0.41709584660478805
